# 그리드 서치

자동반복문 : 최적의 하이퍼파라미터 찾기 위하여  
learning_rate : 경사하강법의 기울기 변화 속도(비용함수의 도함수)와 같음.  
n_estimators : 
min_samples_leaf : 
max_depth : 
max_features : 
이상 5개의 5옵션 반복 => 5^5 => for문 작성할 필요없이 한줄로.  

In [1]:
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

digits = load_digits()

X = digits['data']
Y = digits['target']

model = KNeighborsClassifier()

params = {
    'n_neighbors' : range(1,10),
}

gs = GridSearchCV(model,params).fit(X,Y)
report = pd.DataFrame(gs.cv_results_)
report

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000998,6.306005e-04,0.017288,0.005209,1,{'n_neighbors': 1},0.961111,0.952778,0.966574,0.986072,0.955432,0.964393,0.011838,3
1,0.000998,2.861023e-07,0.016156,0.001596,2,{'n_neighbors': 2},0.961111,0.966667,0.969359,0.977716,0.961003,0.967171,0.006181,1
2,0.000599,4.887334e-04,0.016156,0.000399,3,{'n_neighbors': 3},0.955556,0.958333,0.966574,0.986072,0.966574,0.966622,0.010672,2
3,0.000799,3.992563e-04,0.017757,0.000402,4,{'n_neighbors': 4},0.947222,0.958333,0.966574,0.980501,0.966574,0.963841,0.010946,4
4,0.000798,3.992320e-04,0.018151,0.000746,5,{'n_neighbors': 5},0.947222,0.955556,0.966574,0.980501,0.963788,0.962728,0.011169,5
5,0.000599,4.887724e-04,0.019548,0.003065,6,{'n_neighbors': 6},0.944444,0.958333,0.966574,0.974930,0.952646,0.959386,0.010612,7
6,0.000599,4.888113e-04,0.019348,0.001492,7,{'n_neighbors': 7},0.936111,0.961111,0.969359,0.980501,0.952646,0.959946,0.015059,6
7,0.000798,3.992082e-04,0.017752,0.000747,8,{'n_neighbors': 8},0.936111,0.958333,0.969359,0.977716,0.949861,0.958276,0.014580,8
8,0.000997,1.122341e-06,0.019947,0.002092,9,{'n_neighbors': 9},0.930556,0.952778,0.972145,0.977716,0.949861,0.956611,0.016887,9


In [2]:
gs.best_score_

0.9671711544413494

In [3]:
gs.best_params_

{'n_neighbors': 2}

In [4]:
gs.best_estimator_

KNeighborsClassifier(n_neighbors=2)

In [5]:
gs.best_index_

1

# cv : cross validation 교차검증

데이터 분할 과정에서 비율만 고정하고, tr과 te를 섞어가며 여러번 반복하여 검증하는 방식.  
ex) '1',2,3,4,5 => 1,'2',3,4,5 => 1,2,'3',4,5 => 1,2,3,'4',5 => 1,2,3,4,'5'  
K-Fold 교차 검증 : K겹 검증 => 시간 오래걸림, 과적합 잡을 수 있음. stratified 함께 쓰면 성능 개선 가능  

In [6]:
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

digits = load_digits()

X = digits['data']
Y = digits['target']

model = KNeighborsClassifier()

display(cross_val_score(model,X,Y,cv=5))
display(cross_val_predict(model,X,Y,cv=5))

array([0.94722222, 0.95555556, 0.96657382, 0.98050139, 0.9637883 ])

array([0, 1, 8, ..., 8, 9, 8])

# XGBOOST

빠른 Gradient Boost 
원래 C/C++ -> 컨버팅  
정형데이터 예측 성능 매우 뛰어남  
GPU 지원 가능, 병렬처리 가능  

In [7]:
!pip install xgboost

In [8]:
import warnings
warnings.filterwarnings(action='ignore')

import xgboost as xgb
from sklearn.datasets import load_boston

boston = load_boston()

In [9]:
X = boston['data']
Y = boston['target']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

x_tr, x_te,y_tr,y_te =train_test_split(X,Y,random_state=0)

model = xgb.XGBRegressor()
model.fit(x_tr,y_tr)

p_tr = model.predict(x_tr)
p_te = model.predict(x_te)

r2_score(y_tr,p_tr), r2_score(y_te,p_te)

(0.999999003030776, 0.7476326752660457)

In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_tr,y_tr)
model.score(x_tr,y_tr),model.score(x_te,y_te)

(0.9827724210953349, 0.7682657310780738)

LightGBM

GB 시간을 적게 걸리게 만든 것.  
병렬처리 가능  
수직적 tree 확장(다른 것들은 수평 확장)  
과대적합의 상쇄  
많은 데이터, 빠른 시간, 적은 메모리.  
과적합에 민감, 가적합 나오기 쉽다. => 수가 많은 데이터에 적합함.

In [12]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [14]:
from lightgbm import LGBMRegressor

model = LGBMRegressor().fit(x_tr,y_tr)
model.score(x_tr,y_tr),model.score(x_te,y_te)

(0.9758903113309795, 0.7410313895708159)